In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
data = pd.read_table('taxid10239.nbr', comment='#', header=None, names=["Representative","Neighbor","Host","Selected lineage","Taxonomy name","Segment name"])

In [33]:
data

,Representative,Neighbor,Host,Selected lineage,Taxonomy name,Segment name
0,NC_003663,HQ420896,"human,vertebrates","Poxviridae,Orthopoxvirus,Cowpox virus",Cowpox virus,segment
1,NC_003663,KY463519,"human,vertebrates","Poxviridae,Orthopoxvirus,Cowpox virus",Cowpox virus,segment
2,NC_003663,HQ420897,"human,vertebrates","Poxviridae,Orthopoxvirus,Cowpox virus",Cowpox virus,segment
3,NC_003663,MK035759,"human,vertebrates","Poxviridae,Orthopoxvirus,Cowpox virus",Cowpox virus,segment
4,NC_003663,KY569019,"human,vertebrates","Poxviridae,Orthopoxvirus,Cowpox virus",Cowpox virus,segment
...,...,...,...,...,...,...
260988,NC_054656,MH248947,bacteria,"Yosifvirus,Yosifvirus yosif",Streptomyces phage Yosif,segment
260989,NC_053008,LC553736,bacteria,"Casjensviridae,Zhonglingvirus,Zhonglingvirus S...",Salmonella phage SAP012,segment
260990,NC_053249,MK937611,bacteria,"Zitchvirus,Zitchvirus verity",Gordonia phage Verity,segment
260991,NC_053248,MK937607,bacteria,"Zitchvirus,Zitchvirus zipp",Gordonia phage Zipp,segment


In [3]:
accession = list(data['Neighbor'])
len(accession)

260993

function format

In [ ]:
# Complete code

import pandas as pd
from bs4 import BeautifulSoup
import requests
import re


# the function is designed to get fasta format information
def get_url_info(segment):

    # Using requests function get the id from term
    term_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=nucleotide&api_key=f7aed0f55f83d46ff0249e3e1830661bef08&term=' + ",".join(segment)
    term_info = requests.get(term_url).text
    soup = BeautifulSoup(term_info, 'html.parser')

    # get id
    accession_id = [k.text for k in soup.find_all('id')]

    # get sequencing information from id
    fasta_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&rettype=fasta&id={}&api_key=f7aed0f55f83d46ff0249e3e1830661bef08'.format(','.join(accession_id))
    fasta_info = requests.get(fasta_url).text
    return fasta_info


# this function is designed to write files
def write_file(fasta_info,name_num):
    with open("/home/bio_kang/Learning/Python/requests/data/sequence_{}.fasta".format(name_num), 'w') as f:
        f.write(fasta_info)


# read information from taxid10239.nbr file
data = pd.read_table('taxid10239.nbr', comment='#', header=None, names=["Representative","Neighbor","Host","Selected lineage","Taxonomy name","Segment name"])

# extract 'Neighbor' information
accession = list(data['Neighbor']) 

undownloaded_term = []

for i in range(1,13051):
    segment = accession[(i-1)*20:i*20]
    try:
        fasta_info = get_url_info(segment)
        write_file(fasta_info, i)
    except:
        pattern = re.compile(r'[A-Z]{1,2}\d{5,6}')
        undownloaded_term = undownloaded_term + re.findall(pattern, fasta_info)

In [ ]:
# Complete code

import pandas as pd
from bs4 import BeautifulSoup
import requests

data = pd.read_table('/home/bio_kang/Learning/Python/requests/taxid10239.nbr', comment='#', header=None, names=["Representative","Neighbor","Host","Selected lineage","Taxonomy name","Segment name"]) # get information

accession = list(data['Neighbor']) # extract 'Neighbor' information

# Becauce the RetMax is 20, I slices their length with 20 and the last length is 13
for i in range(13050):
    segment = accession[i*20:(i+1)*20]

    # Using requests function get the id of term
    term_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=nucleotide&api_key=f7aed0f55f83d46ff0249e3e1830661bef08&term=' + ",".join(segment)
    term_info = requests.get(term_url).text
    soup = BeautifulSoup(term_info, 'html.parser')

    accession_id = [k.text for k in soup.find_all('id')] # get id

    # get sequencing information from id
    fasta_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&rettype=fasta&id={}&api_key=f7aed0f55f83d46ff0249e3e1830661bef08'.format(','.join(accession_id))
    fasta_info = requests.get(fasta_url).text

    # get fasta file
    with open("/home/bio_kang/Learning/Python/requests/data/sequence_{}.fasta".format(i), 'w') as f:
        f.write(fasta_info)


check processes

In [ ]:
renew = []
with open('/home/bio_kang/data/virus_data/complement_data/omitted') as f:
    omitted_info_set = f.read().split('\n')
    omitted_info_set.pop(-1)
for omitted_info in omitted_info_set:
    pattern = re.compile(r'\d+')
    renew = renew + re.findall(pattern,omitted_info)

omitted = []
for num in renew:
    omitted = omitted + accession[(int(num)-1)*20:int(num)*20]

omitted.pop(25) # delete NCBI_GENOMES

for num in range(20):
    term_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=nucleotide&api_key=bb9bbede507a3504dc8923c13b4e94080c08&term=' + ','.join(omitted[num*20: (num+1)*20])
    term_info = requests.get(term_url).text
    soup = BeautifulSoup(term_info, 'html.parser')

    accession_id = [k.text for k in soup.find_all('id')]

    fasta_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&rettype=fasta&id={}&api_key=bb9bbede507a3504dc8923c13b4e94080c08'.format(','.join(accession_id))
    fasta_info = requests.get(fasta_url).text

    with open('/home/bio_kang/data/virus_data/complement_data/complement_{}.fasta'.format(num), 'w') as f:
        f.write(fasta_info)

In [81]:
seg = 'EF537008'
term_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=nucleotide&api_key=bb9bbede507a3504dc8923c13b4e94080c08&term=' + seg
term_info = requests.get(term_url).text
soup = BeautifulSoup(term_info, 'html.parser')
id = soup.find_all('id')
print(id)

[<id>146230032</id>]


In [47]:
import re
pattern = re.compile(r'[a-z]{8}_\d+.\w{5}')
ls = re.search(pattern,'/home/bio_kang/data/virus_data/sequence_3307.fasta')
print(ls)

<re.Match object; span=(31, 50), match='sequence_3307.fasta'>


In [70]:
term_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=nucleotide&api_key=bb9bbede507a3504dc8923c13b4e94080c08&term=' + "NCBI_GENOMES"
term_info = requests.get(term_url).text
pattern = re.compile(r'\d{10}')
id = re.findall(pattern, term_info)
fasta_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&rettype=fasta&id={}&api_key=bb9bbede507a3504dc8923c13b4e94080c08'.format(','.join(id))
fasta_info = requests.get(fasta_url).text
with open('/home/bio_kang/data/virus_data/complement_data/NCBI_GENOMES', 'w') as f:
    f.write(fasta_info)